In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [7]:
def make_models(tickerSymbol):
 

    # Get data on the ticker
    tickerData = yf.Ticker(tickerSymbol)

    # Get the historical prices for this ticker
    tickerDf = tickerData.history(period='1d', start='2015-01-01', end='2023-04-28')

    # Calculate the EMA with a window of 20 days
    ema = tickerDf['Close'].ewm(span=20, adjust=False).mean()

    # Merge the EMA with the original DataFrame
    tickerDf = pd.concat([tickerDf, ema.rename('EMA')], axis=1)

    # Define the feature and target variables
    data = tickerDf[['Close', 'EMA']]
    target = tickerDf['Close']

    # Normalize the data using MinMaxScaler
    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(data)


    # Define the number of time steps for the LSTM model
    n_steps = 1000

    # Create a function to split the data into input/output samples
    def prepare_data(data, n_steps):
        X, y = [], []
        for i in range(len(data)):
            end_ix = i + n_steps
            if end_ix > len(data)-1:
                break
            seq_x, seq_y = data[i:end_ix, :], data[end_ix, 0]
            X.append(seq_x)
            y.append(seq_y)
        return np.array(X), np.array(y)

    # Prepare the data for the LSTM model
    X, y = prepare_data(data_scaled, n_steps)

    # Split the data into training and testing sets
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Define the LSTM model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(n_steps, 2)))
    model.add(LSTM(units=50))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Fit the model to the training data
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

    # Evaluate the model on the testing data
    mse = model.evaluate(X_test, y_test, verbose=0)
    print('Mean Squared Error:', mse)

    # Save the trained model to a file
    model.save(f'models1000\\{tickerSymbol.upper()}_model.h5')

In [6]:
tickers = ["TSLA","TM", "MBGYY","F","GM","HMC"]
for ticker in tickers:
    make_models(ticker)

Mean Squared Error: 0.000674326263833791
Mean Squared Error: 0.0011096640955656767
Mean Squared Error: 0.001282067270949483
Mean Squared Error: 0.0009658022900111973
Mean Squared Error: 0.0007699995767325163
Mean Squared Error: 0.0011413744650781155
